# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the city details from Part1 output csv file
city_weather_df = pd.read_csv("output_data/cities.csv")
city_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ca Mau,9.1769,105.1500,83.37,66,99,12.35,VN,1639706254
1,Ukiah,39.1502,-123.2078,44.73,95,9,2.82,US,1639706278
2,Hailar,49.2000,119.7000,-6.90,90,17,9.31,CN,1639706278
3,Bredasdorp,-34.5322,20.0403,57.27,69,7,17.43,ZA,1639706278
4,Praia da Vitória,38.7333,-27.0667,62.85,88,75,26.96,PT,1639706278
...,...,...,...,...,...,...,...,...,...
567,Juba,4.8517,31.5825,75.85,55,97,3.49,SS,1639707130
568,Kisanga,-7.4362,37.7034,71.82,96,100,1.10,TZ,1639707130
569,Enid,36.3956,-97.8784,39.58,58,1,8.05,US,1639707130
570,Bathsheba,13.2167,-59.5167,79.41,69,40,24.16,BB,1639707131


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)

# Store Humidity into weight
humidity = city_weather_df["Humidity"].astype(float)


In [4]:
# Configure map, set display and zoom size
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(25, 13), zoom_level=2)

# Create weather Heatmap layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=330,
                                 point_radius = 5)

# Add weather heatmap layer
fig.add_layer(heat_layer)

# Display the weather heatmap
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Export the weather heatmap to html file
embed_minimal_html('output_images/Weather Map.html', views=[fig], title='Weather Map')

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Select only cities (i.e. rows) that meet the predefined ideal weather 
cities_with_ideal_weather = city_weather_df.loc[(city_weather_df['Max Temp']>65) 
                                               & (city_weather_df['Max Temp']<85) 
                                               & (city_weather_df['Wind Speed']<10)  
                                               & (city_weather_df['Cloudiness']==0)].dropna()
cities_with_ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,San Miguel de Tucumán,-26.8241,-65.2226,73.40,83,0,6.91,AR,1639706284
36,Villa Carlos Paz,-31.4241,-64.4978,65.84,37,0,3.00,AR,1639706288
112,Seddon,-37.8068,144.8916,81.48,36,0,5.01,AU,1639706431
197,Ambilobe,-13.2000,49.0500,71.80,90,0,3.36,MG,1639706529
255,Villarrica,-25.7500,-56.4333,80.55,42,0,7.90,PY,1639706606
341,Salalah,17.0151,54.0924,71.69,60,0,6.91,OM,1639706756
403,Paredon,16.0333,-93.8667,79.09,73,0,4.74,MX,1639706896
413,Vao,-22.6667,167.4833,74.95,63,0,9.01,NC,1639706640


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a new hotel_df with new empty column to store hotel name
hotel_df = cities_with_ideal_weather[['City','Country',"Lat", "Lng"]]
hotel_df.insert(4, "Hotel Name", "") 
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,San Miguel de Tucumán,AR,-26.8241,-65.2226,
36,Villa Carlos Paz,AR,-31.4241,-64.4978,
112,Seddon,AU,-37.8068,144.8916,
197,Ambilobe,MG,-13.2000,49.0500,
255,Villarrica,PY,-25.7500,-56.4333,
341,Salalah,OM,17.0151,54.0924,
403,Paredon,MX,16.0333,-93.8667,
413,Vao,NC,-22.6667,167.4833,


In [8]:
# Set params dictionary to update each iteration within a defined radius of 5000 metres

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng of the selected cities in hotel_df dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and the lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)

    # convert to json
    hotel_name = hotel_name.json()

    # Since some data may be missing,incorporated here a try-except to skip missing data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing data field/result... skipping.")
        pass

hotel_df

C:\Users\snoop\anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing data field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
25,San Miguel de Tucumán,AR,-26.8241,-65.2226,Sheraton Tucuman Hotel
36,Villa Carlos Paz,AR,-31.4241,-64.4978,Hipocampus Hotel & Spa
112,Seddon,AU,-37.8068,144.8916,Pan Pacific Melbourne
197,Ambilobe,MG,-13.2000,49.0500,Hotel National
255,Villarrica,PY,-25.7500,-56.4333,Hotel Ybytyruzu
341,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
403,Paredon,MX,16.0333,-93.8667,
413,Vao,NC,-22.6667,167.4833,OURE TERA beach resort


In [9]:
# Clean data to remove missing hotel name information
hotel_df = hotel_df.drop(hotel_df.index[hotel_df['Hotel Name'] ==''], inplace = False)
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,San Miguel de Tucumán,AR,-26.8241,-65.2226,Sheraton Tucuman Hotel
36,Villa Carlos Paz,AR,-31.4241,-64.4978,Hipocampus Hotel & Spa
112,Seddon,AU,-37.8068,144.8916,Pan Pacific Melbourne
197,Ambilobe,MG,-13.2000,49.0500,Hotel National
255,Villarrica,PY,-25.7500,-56.4333,Hotel Ybytyruzu
341,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
413,Vao,NC,-22.6667,167.4833,OURE TERA beach resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [11]:
# Add marker layer ontop of heat map

# Create a map using coordinates to set markers
marker_locations = hotel_df[["Lat", "Lng"]]

# Create a marker_layer using the hotel_info to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)

# Add map layers
fig = gmaps.figure(center=(9, 15), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(markers)


In [12]:
# Export map to html file and save to output_images folder 
# The HTML file can only be viewed in the browser, NOT via jupypter notebook. Refer to the README file for more details.
# The map can also be manually downloaded by clicking on the download button on the top left corner of the graph.
embed_minimal_html('output_images/Hotel Map.html', views=[fig], title='Hotel Map')

In [13]:
# Display map figure on screen
fig

Figure(layout=FigureLayout(height='420px'))